In [1]:
import pandas as pd
import numpy as np

# Adjust pandas display settings for better viewing
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("--- Loading all data files ---")
try:
    # Main data
    train_df = pd.read_parquet('data/train_data.parquet')
    test_df = pd.read_parquet('data/test_data.parquet')

    # Supplemental data
    add_trans_df = pd.read_parquet('data/add_trans.parquet')
    add_event_df = pd.read_parquet('data/add_event.parquet')
    offer_meta_df = pd.read_parquet('data/offer_metadata.parquet')

    # Supporting files
    data_dict_df = pd.read_csv('data/data_dictionary.csv')
    submission_df = pd.read_csv('data/685404e30cfdb_submission_template.csv')

    print("All datasets loaded successfully!")

except FileNotFoundError as e:
    print(f"Error: {e}. Please ensure all files are in the correct directory.")

--- Loading all data files ---
All datasets loaded successfully!


In [2]:
# --- Check initial data types of join keys ---
print("--- Initial dtypes of 'id2' ---")
print(f"train_df: {train_df['id2'].dtype}")
print(f"add_trans_df: {add_trans_df['id2'].dtype}")
print(f"add_event_df: {add_event_df['id2'].dtype}")

print("\n--- Initial dtypes of 'id3' ---")
print(f"train_df: {train_df['id3'].dtype}")
print(f"offer_meta_df: {offer_meta_df['id3'].dtype}")
print(f"add_event_df: {add_event_df['id3'].dtype}")

--- Initial dtypes of 'id2' ---
train_df: object
add_trans_df: int32
add_event_df: int32

--- Initial dtypes of 'id3' ---
train_df: object
offer_meta_df: int32
add_event_df: object


In [3]:
print("--- Unifying key column data types ---")

# List of dataframes to update
dfs_to_update = {
    'train_df': train_df,
    'test_df': test_df,
    'add_trans_df': add_trans_df,
    'add_event_df': add_event_df,
    'offer_meta_df': offer_meta_df
}

# Convert 'id2' and 'id3' to numeric (int64) where they exist
for name, df in dfs_to_update.items():
    if 'id2' in df.columns:
        df['id2'] = pd.to_numeric(df['id2'], errors='coerce').astype('Int64')
        print(f"Converted 'id2' in {name} to {df['id2'].dtype}")
    if 'id3' in df.columns:
        df['id3'] = pd.to_numeric(df['id3'], errors='coerce').astype('Int64')
        print(f"Converted 'id3' in {name} to {df['id3'].dtype}")

print("\n--- Verification of new dtypes ---")
print("\n--- Dtypes of 'id2' after conversion ---")
print(f"train_df: {train_df['id2'].dtype}")
print(f"test_df: {test_df['id2'].dtype}")
print(f"add_trans_df: {add_trans_df['id2'].dtype}")
print(f"add_event_df: {add_event_df['id2'].dtype}")


print("\n--- Dtypes of 'id3' after conversion ---")
print(f"train_df: {train_df['id3'].dtype}")
print(f"test_df: {test_df['id3'].dtype}")
print(f"offer_meta_df: {offer_meta_df['id3'].dtype}")
print(f"add_event_df: {add_event_df['id3'].dtype}")

--- Unifying key column data types ---
Converted 'id2' in train_df to Int64
Converted 'id3' in train_df to Int64
Converted 'id2' in test_df to Int64
Converted 'id3' in test_df to Int64
Converted 'id2' in add_trans_df to Int64
Converted 'id2' in add_event_df to Int64
Converted 'id3' in add_event_df to Int64
Converted 'id3' in offer_meta_df to Int64

--- Verification of new dtypes ---

--- Dtypes of 'id2' after conversion ---
train_df: Int64
test_df: Int64
add_trans_df: Int64
add_event_df: Int64

--- Dtypes of 'id3' after conversion ---
train_df: Int64
test_df: Int64
offer_meta_df: Int64
add_event_df: Int64


In [4]:
print("--- Starting initial data cleaning ---")

# Clean offer_meta_df by dropping completely empty columns
offer_meta_df.drop(columns=['f377', 'id11'], inplace=True, errors='ignore')
print("Cleaned 'offer_meta_df'.")

# Identify and drop columns with over 95% missing values in the training set
missing_val_percent = train_df.isnull().sum() / len(train_df) * 100
cols_to_drop = missing_val_percent[missing_val_percent > 95].index
train_df.drop(columns=cols_to_drop, inplace=True, errors='ignore')
test_df.drop(columns=cols_to_drop, inplace=True, errors='ignore')

print(f"\nDropped {len(cols_to_drop)} columns with >95% missing values: {cols_to_drop.tolist()}")
print(f"Remaining columns in train_df: {train_df.shape[1]}")

# --- Target Variable Analysis ---
# Convert 'y' to numeric and analyze its distribution
train_df['y'] = pd.to_numeric(train_df['y'], errors='coerce')
y_dist_percent = train_df['y'].value_counts(normalize=True).mul(100).round(2)

print("\n--- Target Variable 'y' Distribution ---")
print(y_dist_percent)


--- Starting initial data cleaning ---
Cleaned 'offer_meta_df'.

Dropped 32 columns with >95% missing values: ['f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f33', 'f34', 'f37', 'f64', 'f66', 'f70', 'f79', 'f80', 'f84', 'f88', 'f92', 'f112', 'f120', 'f122', 'f135', 'f136', 'f154', 'f176', 'f189', 'f205', 'f220', 'f221', 'f360']
Remaining columns in train_df: 340

--- Target Variable 'y' Distribution ---
y
0    95.19
1     4.81
Name: proportion, dtype: float64


In [5]:
print("--- Engineering features from transaction data ---")

# Convert date column to datetime objects for recency calculation
add_trans_df['f370'] = pd.to_datetime(add_trans_df['f370'], errors='coerce')

# Find the most recent transaction date in the entire dataset to calculate recency
last_transaction_date = add_trans_df['f370'].max()
print(f"The last transaction date in the dataset is: {last_transaction_date.date()}")


# Group by customer ('id2') and create aggregated features
agg_trans_df = add_trans_df.groupby('id2').agg(
    # Transaction amount features
    avg_trans_amt=('f367', 'mean'),
    sum_trans_amt=('f367', 'sum'),
    max_trans_amt=('f367', 'max'),
    
    # Transaction count
    total_transactions=('id2', 'count'),
    
    # Unique transaction types (e.g., PBR, PR, PGC)
    nunique_trans_types=('f368', 'nunique'),
    
    # Recency Calculation: Days since the customer's last transaction
    days_since_last_transaction=('f370', lambda x: (last_transaction_date - x.max()).days)
)

print(f"\nCreated new aggregated transaction dataframe with shape: {agg_trans_df.shape}")


# Merge the new transaction features into the main dataframes
train_df = pd.merge(train_df, agg_trans_df, on='id2', how='left')
test_df = pd.merge(test_df, agg_trans_df, on='id2', how='left')

print("\nMerged new transaction features into train and test sets.")
print(f"New shape of train_df: {train_df.shape}")
print(f"New shape of test_df: {test_df.shape}")

--- Engineering features from transaction data ---
The last transaction date in the dataset is: 2023-11-03

Created new aggregated transaction dataframe with shape: (194115, 6)

Merged new transaction features into train and test sets.
New shape of train_df: (770164, 346)
New shape of test_df: (369301, 345)


In [6]:
# --- Display the first 5 rows with the new features ---
train_df.head()

,id1,id2,id3,id4,id5,y,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f35,f36,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f65,f67,f68,f69,f71,f72,f73,f74,f75,f76,f77,f78,f81,f82,f83,f85,f86,f87,f89,f90,f91,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f113,f114,f115,f116,f117,f118,f119,f121,f123,f124,f125,f126,f127,f128,f129,f130,f131,f132,f133,f134,f137,f138,f139,f140,f141,f142,f143,f144,f145,f146,f147,f148,f149,f150,f151,f152,f153,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164,f165,f166,f167,f168,f169,f170,f171,f172,f173,f174,f175,f177,f178,f179,f180,f181,f182,f183,f184,f185,f186,f187,f188,f190,f191,f192,f193,f194,f195,f196,f197,f198,f199,f200,f201,f202,f203,f204,f206,f207,f208,f209,f210,f211,f212,f213,f214,f215,f216,f217,f218,f219,f222,f223,f224,f225,f226,f227,f228,f229,f230,f231,f232,f233,f234,f235,f236,f237,f238,f239,f240,f241,f242,f243,f244,f245,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284,f285,f286,f287,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297,f298,f299,f300,f301,f302,f303,f304,f305,f306,f307,f308,f309,f310,f311,f312,f313,f314,f315,f316,f317,f318,f319,f320,f321,f322,f323,f324,f325,f326,f327,f328,f329,f330,f331,f332,f333,f334,f335,f336,f337,f338,f339,f340,f341,f342,f343,f344,f345,f346,f347,f348,f349,f350,f351,f352,f353,f354,f355,f356,f357,f358,f359,f361,f362,f363,f364,f365,f366,avg_trans_amt,sum_trans_amt,max_trans_amt,total_transactions,nunique_trans_types,days_since_last_transaction
0,1366776_189706075_16-23_2023-11-02 22:22:00.042,1366776,189706075,2023-11-02 22:22:00.042,2023-11-02,0,1.0,None,None,None,None,None,None,None,None,13.0,27.0,None,2.0,0.0,0.0,0.0,2.0,0.0,27.1294,None,418.2614125177312,4.868919391030028,None,None,None,16.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1114698.0,0.0,0.0,0.0,0.0,0.0,1114698.0,15174577.0,0.0,0.0,0.0,0.0,0.0,0.0,15174577.0,0.2156862745098039,None,None,None,None,1.0,0.0,0.0,0.0,0.0,0.0,1.0,7.0,121.0,63.0,0.0,23.0,104.0,23.0,13.0,0.0,354.0,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0,0.0,None,None,None,None,None,None,None,1.0,4.0,1.0,2.0,3.0,0.0,0.0,0.0125,0.0555555555555555,0.0005313496280552,0.0117647058823529,0.0157894736842105,0.004594820384294,0.0070351758793969,1.5311100959342168,0.0909090909090909,0.3636363636363636,0.1818181818181818,0.2727272727272727,0.0,0.0,445.5172883744984,4.868919391030028,638.7711506045847,3.7321319661472296,0.0076223219950082,0.0083770755109508,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,379.0,360.07881805753976,379.0,372.3613036362676,1.052547334065723,1.017828641963874,None,None,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,29.0,1.0,None,2.0,None,None,1.0,28.0,29.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1.0,0.0,0.0,0.0,0.102967563837129,0.1399675779235134,0.1139605073678811,0.0952545856699211,0.0800295493720758,0.7356529659561349,1.2282112563054637,1.1963781750390508,244278.0,1388862.0,1388862.0,None,None,5,80458,1.0,3.0,0.0,Phase_1,None,None,None,-9999.0,0.0,28.0,0.0,0.0,337.0,0.0,0.0,NaN,NaN,NaN,<NA>,NaN,NaN
1,1366776_89227_16-23_2023-11-01 23:51:24.999,1366776,89227,2023-11-01 23:51:24.999,2023-11-01,0,1.0,None,None,None,None,None,None,None,None,13.0,2

In [7]:
print("--- Engineering features from event data ---")

# Convert event timestamp column to datetime objects
add_event_df['id4'] = pd.to_datetime(add_event_df['id4'], errors='coerce')

# Find the most recent event date in the entire dataset
last_event_date = add_event_df['id4'].max()
print(f"The last event date in the dataset is: {last_event_date.date()}")

# Group by customer ('id2') and create aggregated features
agg_event_df = add_event_df.groupby('id2').agg(
    # Event counts
    total_events=('id2', 'count'),
    
    # Unique offer interactions
    nunique_offers_interacted=('id3', 'nunique'),
    
    # Unique event types (e.g., Tiles, OffersTab)
    nunique_event_types=('id6', 'nunique'),
    
    # Recency Calculation: Days since the customer's last event
    days_since_last_event=('id4', lambda x: (last_event_date - x.max()).days)
)

print(f"\nCreated new aggregated event dataframe with shape: {agg_event_df.shape}")

# Merge the new event features into the main dataframes
train_df = pd.merge(train_df, agg_event_df, on='id2', how='left')
test_df = pd.merge(test_df, agg_event_df, on='id2', how='left')

print("\nMerged new event features into train and test sets.")
print(f"New shape of train_df: {train_df.shape}")
print(f"New shape of test_df: {test_df.shape}")

--- Engineering features from event data ---
The last event date in the dataset is: 2023-11-03

Created new aggregated event dataframe with shape: (428195, 4)

Merged new event features into train and test sets.
New shape of train_df: (770164, 350)
New shape of test_df: (369301, 349)


In [8]:
# --- Display the first 5 rows with all new features ---
# We select a subset of original and all new columns for readability
new_feature_cols = ['avg_trans_amt', 'total_transactions', 'days_since_last_transaction', 
                    'total_events', 'nunique_offers_interacted', 'days_since_last_event']
train_df[['id1', 'id2', 'id3'] + new_feature_cols].head()

,id1,id2,id3,avg_trans_amt,total_transactions,days_since_last_transaction,total_events,nunique_offers_interacted,days_since_last_event
0,1366776_189706075_16-23_2023-11-02 22:22:00.042,1366776,189706075,NaN,<NA>,NaN,<NA>,NaN,NaN
1,1366776_89227_16-23_2023-11-01 23:51:24.999,1366776,89227,NaN,<NA>,NaN,<NA>,NaN,NaN
2,1366776_35046_16-23_2023-11-01 00:30:59.797,1366776,35046,NaN,<NA>,NaN,<NA>,NaN,NaN
3,1366776_6275451_16-23_2023-11-02 22:21:32.261,1366776,6275451,NaN,<NA>,NaN,<NA>,NaN,NaN
4,1366776_78053_16-23_2023-11-02 22:21:34.799,1366776,78053,NaN,<NA>,NaN,<NA>,NaN,NaN


In [9]:
print("--- Merging offer metadata ---")

# Merge the offer metadata into the main dataframes
# A left join ensures we keep all rows from train_df and test_df
train_df = pd.merge(train_df, offer_meta_df, on='id3', how='left')
test_df = pd.merge(test_df, offer_meta_df, on='id3', how='left')

print("Offer metadata merged successfully.")

--- Merging offer metadata ---
Offer metadata merged successfully.


In [10]:
# --- Display info for the fully merged dataframe ---
print("\n--- Info for train_df after all merges ---")
train_df.info()


--- Info for train_df after all merges ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770164 entries, 0 to 770163
Columns: 359 entries, id1 to id13
dtypes: Int64(4), float64(10), int64(1), object(344)
memory usage: 2.1+ GB


In [ ]:
print("--- Starting final data preparation ---")

# --- 1. Separate target and identifiers ---
# Convert target 'y' to numeric and handle potential NaNs
y = pd.to_numeric(train_df['y'], errors='coerce').fillna(0)

# Drop unnecessary columns. We keep 'id1' in test_df for the submission file.
train_df = train_df.drop(columns=['id1', 'y', 'id4', 'id5'])
test_df = test_df.drop(columns=['id1', 'id4', 'id5'])

print("Separated target variable and dropped identifiers.")

# --- 2. Convert object columns to category for efficiency ---
categorical_cols = train_df.select_dtypes(include='object').columns.tolist()

for col in categorical_cols:
    # Combine categories from both train and test to prevent errors
    all_categories = pd.concat([train_df[col], test_df[col]], axis=0).dropna().unique()
    train_df[col] = pd.Categorical(train_df[col], categories=all_categories)
    test_df[col] = pd.Categorical(test_df[col], categories=all_categories)

print("Converted object columns to category type.")


# --- 3. Impute Missing Values ---
# Impute numerical columns with the median from the training set
numerical_cols = train_df.select_dtypes(include=np.number).columns.tolist()
for col in numerical_cols:
    median_val = train_df[col].median()
    train_df[col].fillna(median_val, inplace=True)
    test_df[col].fillna(median_val, inplace=True)

# Impute categorical columns with the mode from the training set
for col in categorical_cols:
    mode_val = train_df[col].mode()[0]
    train_df[col].fillna(mode_val, inplace=True)
    test_df[col].fillna(mode_val, inplace=True)

print("Filled all remaining missing values.")


# --- 4. One-Hot Encode Categorical Features ---
# This converts categories into a numerical format for the model
train_df = pd.get_dummies(train_df, columns=categorical_cols, dummy_na=False)
test_df = pd.get_dummies(test_df, columns=categorical_cols, dummy_na=False)

# Align columns to ensure train and test sets have the exact same features
train_df, test_df = train_df.align(test_df, join='inner', axis=1)

print("Applied one-hot encoding.")


# --- 5. Scale Numerical Features ---
# Update numerical_cols list after one-hot encoding
numerical_cols = train_df.select_dtypes(include=np.number).columns.tolist()

scaler = StandardScaler()
train_df[numerical_cols] = scaler.fit_transform(train_df[numerical_cols])
test_df[numerical_cols] = scaler.transform(test_df[numerical_cols])

print("Applied feature scaling.")


# --- 6. Final Verification ---
print(f"\nFinal shape of training data (X): {train_df.shape}")
print(f"Final shape of test data (X_test): {test_df.shape}")
print(f"Total remaining missing values in train_df: {train_df.isnull().sum().sum()}")
print(f"Total remaining missing values in test_df: {test_df.isnull().sum().sum()}")

--- Starting final data preparation ---
Separated target variable and dropped identifiers.
Converted object columns to category type.


/var/folders/rr/416hfp154r52dvpst01zsqpr0000gn/T/ipykernel_60356/740303584.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df[col].fillna(median_val, inplace=True)
/var/folders/rr/416hfp154r52dvpst01zsqpr0000gn/T/ipykernel_60356/740303584.py:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always

Filled all remaining missing values.
